# Software Design Patterns

In [ ]:
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
import numpy as np

If the last line didn't work, please install matplotlib, e.g. by
typing ``pip3 install --user install matplotlib``.

# Creational patterns

## Factory method pattern

In [ ]:
class Histogram(object):
    def __init__(self, bin_contents=None, bin_edges=None, data=None, nbins=10):
        """ Initialize histogram either by directly giving bin contents and edges
        or from data.
        
        Args:
            bin_contents: List of bin contents
            bin_edges: List of bin edges (n+1 values for n bins)
            data: List of data points that will be automatically binned
            nbins: Number of bins to bin data in, if bin_edges is not given
        """
        if bin_contents is not None and bin_edges is not None:
            self.bin_contents = bin_contents
            self.bin_edges = bin_edges
        elif data is not None:
            if bin_edges is not None:
                self.bin_contents, self.bin_edges = np.histogram(data, bins=bin_edges)
            else:
                self.bin_contents, self.bin_edges = np.histogram(data, bins=nbins)

h1 = Histogram(bin_contents=[1, 2, 3], bin_edges=[1, 2, 3, 4])
h2 = Histogram(data=[1, 5, 7, 8, 4, 2, 2, 3, 4], nbins=5)

**Exercise**: Simplify the constructor using the factory method pattern. 
How can you recreate the two objects from before?

<html>
<details>
<summary style="color:orange; font-weight: bold;"> Click me for a hint </summary>
    Change the <code>__init__</code> constructor so that it only takes <code>bin_contents</code> and <code>bin_edges</code>.
Create a new classmethod <code>from_data</code> that handles the construction from data.    
</details>
</html>

<html>
<details>
<summary style="color:red; font-weight: bold;"> Click me for the solution </summary>
Define the classmethod as follows.

```python
class Histogram(object):
    def __init__(self, bin_contents, bin_edges):
        self.bin_contents = bin_contents
        self.bin_edges = bin_edges
    
    @classmethod
    def from_data(cls, data, bin_edges=None, nbins=10):
        if bin_edges is not None:
            bin_contents, bin_edges = np.histogram(data, bins=bin_edges)
        else:
            bin_contents, bin_edges = np.histogram(data, bins=nbins)
        return cls(bin_contents=bin_contents, bin_edges=bin_edges)

h1 = Histogram(bin_contents=[1, 2, 3], bin_edges=[1, 2, 3, 4])
h2 = Histogram.from_data(data=[1, 5, 7, 8, 4, 2, 2, 3, 4], nbins=5)
```
</details>
</html>

## Builder pattern

Let's look at the following example of a class for plotting (you can run the next two cells to get an idea about what it does).

In [ ]:
class ComplexPlot(object):
    def __init__(self, xvalues, yvalues, colors, labels, hlines, vlines, hlinecolors, vlinecolors):
        self.xvalues = xvalues
        self.yvalues = yvalues
        self.colors = colors
        self.labels = labels
        self.hlines = hlines
        self.vlines = vlines
        self.hlinecolors = hlinecolors
        self.vlinecolors = vlinecolors
    
    def plot(self):
        fig, ax = plt.subplots()
        for i in range(len(self.xvalues)):
            ax.plot(self.xvalues[i], self.yvalues[i], color=self.colors[i], label=self.labels[i])
        for i in range(len(self.hlines)):
            ax.axhline(self.hlines[i], color=self.hlinecolors[i])
        for i in range(len(self.vlines)):
            ax.axvline(self.vlines[i], color=self.vlinecolors[i])
        ax.legend()
        return ax

In [ ]:
c = ComplexPlot(
    xvalues=[[1, 2, 3], [1, 2, 3]],
    yvalues=[[1, 2, 3], [4, 5, 6]],
    colors=["black", "red"],
    labels=["label 1", "label 2"],
    hlines=[1],
    vlines=[2],
    hlinecolors=["yellow"],
    vlinecolors=["gray"]
)
c.plot()

**Exercise**: 

1. Make use of the *builder pattern* to simplify the above class and to make it more flexible. 
2. Reproduce the plot with your new class

<html>
<details>
<summary style="color:red; font-weight: bold;"> Click me for the solution </summary>
 
```python
class ComplexPlot(object):
    def __init__(self):
        self.xvalues = []
        self.yvalues = []
        self.colors = []
        self.labels = []
        self.hlines = []
        self.vlines = []
        self.hlinecolors = []
        self.vlinecolors = []
    
    def add_dataset(self, x, y, color="black", label=""):
        self.xvalues.append(x)
        self.yvalues.append(y)
        self.colors.append(color)
        self.labels.append(label)
    
    def add_hline(self, y, color="black"):
        self.hlines.append(y)
        self.hlinecolors.append(color)
    
    def add_vline(self, y, color="black"):
        self.hlines.append(y)
        self.hlinecolors.append(color)
    
    def plot(self):
        fig, ax = plt.subplots()
        for i in range(len(self.xvalues)):
            ax.plot(self.xvalues[i], self.yvalues[i], color=self.colors[i], label=self.labels[i])
        for i in range(len(self.hlines)):
            ax.axhline(self.hlines[i], color=self.hlinecolors[i])
        for i in range(len(self.vlines)):
            ax.axvline(self.vlines[i], color=self.vlinecolors[i])
        ax.legend()
        return ax

c = ComplexPlot()
c.add_dataset([1, 2, 3], [1, 2, 3], label="label 1")
c.add_dataset([1, 2, 3], [4, 5, 6], color="red", label="label 2")
c.add_hline(1, color="yellow")
c.add_vline(2, color="gray")
c.plot()
```
    
</details>
</html>

# Structural patterns

## Adapter

You have a nice setup for dealing with lamps:

In [ ]:
class Lamp(ABC):
    """ Interface of my lamps. """
    @abstractmethod
    def toggle(self):
        pass

class MyOldLamp(Lamp):
    """ This is one of my specific lamps. """
    def __init__(self):
        self._turned_on = False
    
    def get_state(self):
        # (Note for advanced python users: 
        # Accessor function because lecture didn't introduce
        # the '@property' decorator)
        return self._turned_on
    
    def toggle(self):
        self._turned_on = not self._turned_on


def report_and_switch_on_lamp(lamp: Lamp):
    """ This function prints whether the lamp is lit
    and if not, flips the switch. """
    if lamp.get_state():
        print("Lamp is lit")
    else:
        print("Lamp is dark")
    lamp.toggle()

Let's try it!

In [ ]:
mol = MyOldLamp()

report_and_switch_on_lamp(mol)
report_and_switch_on_lamp(mol)

Let's try with a new lamp:

In [ ]:
class MyNewLamp():
    def __init__(self):
        self.turned_on = False
    
    def get_state(self):
        return self.turned_on
    
    def flip_switch(self):
        self.turned_on = not self.turned_on

In [ ]:
mnl = MyNewLamp()
report_and_switch_on_lamp(mnl)
report_and_switch_on_lamp(mnl)  # Ooouch

That didn't work!

**Exercise**: Create an adapter class for ``MyNewLamp``

<html>
<details>
<summary style="color:red; font-weight: bold;"> Click me for the solution </summary>
 
```python
class MyNewLampAdapter(Lamp):
    def __init__(self, lamp: MyNewLamp):
        self._lamp = lamp
    
    def get_state(self):
        return self._lamp.get_state()
    
    def toggle(self):
        self._lamp.flip_switch()

mnl = MyNewLampAdapter(MyNewLamp())
report_and_switch_on_lamp(mnl)
report_and_switch_on_lamp(mnl)
```
    
</details>
</html>

# Behavioral patterns

## Template method

In [ ]:
class MLModel():
    def __init__(self, model="BDT"):
        self.model = model
    
    def prepare_features(self):
        if self.model == "BDT":
            print("Preparing features for BDT")
        elif self.model == "RandomForest":
            print("Preparing features for RandomForest")
        else:
            raise ValueError("Invalid model")
    
    def train(self):
        if self.model == "BDT":
            print("Training BDT model")
        elif self.model == "RandomForest":
            print("Training RandomForest")
        else:
            raise ValueError("Invalid model")
    
    def validate(self):
        print("Your model is doing okayish...")
    
    
    def run_all(self):
        self.prepare_features()
        self.train()
        self.validate()

m1 = MLModel(model="BDT")
m1.run_all()

m2 = MLModel(model="RandomForest")
m2.run_all()

**Exercise**: What are the problems here? How can you solve them with the template method pattern?

<html>
<details>
<summary style="color:orange; font-weight: bold;"> Click me for a hint </summary>
    Make the <code>prepare_features</code> and <code>train</code> methods of <code>MLModel</code> abstract.
    Create two subclasses that implement them.
</details>
</html>

<html>
<details>
<summary style="color:orange; font-weight: bold;"> Click me for a hint </summary>
Your code for the new class might look like this:

```python
class MLModel(ABC):
    
    @abstractmethod
    def prepare_features(self):
        pass
    
    @abstractmethod
    def train(self):
        pass
    
    def validate(self):
        print("Your model is doing okayish...")
    
    
    def run_all(self):
        self.prepare_features()
        self.train()
        self.validate()
```
</details>
</html>

<html>
<details>
<summary style="color:orange; font-weight: bold;"> Click me for a hint </summary>
Your code for one of the subclasses might look like this:

```python
class BDT(MLModel):
    def prepare_features(self):
        print("Preparing features for BDT")
    
    def train(self):
        print("Training BDT")
```
</details>
</html>

<html>
<details>
<summary style="color:red; font-weight: bold;"> Click me for the solution </summary>
Full solution:

```python
class MLModel(ABC):
    
    @abstractmethod
    def prepare_features(self):
        pass
    
    @abstractmethod
    def train(self):
        pass
    
    def validate(self):
        print("Your model is doing okayish...")
    
    
    def run_all(self):
        self.prepare_features()
        self.train()
        self.validate()
        
class BDT(MLModel):
    def prepare_features(self):
        print("Preparing features for BDT")
    
    def train(self):
        print("Training BDT")

class RandomForest(MLModel):
    def prepare_features(self):
        print("Preparing features for RandomForest")
    
    def train(self):
        print("Training RandomForest")

m1 = BDT()
m1.run_all()

m2 = RandomForest()
m2.run_all()
```
</details>
</html>

## Strategy pattern

**Exercise**: Look at the strategy pattern example in the slides. Build up a similar ``MyAnalysis`` class that 
takes two machine learning models ``ml_model_1`` and ``ml_model_2``. Use the example from the last exercise
to initialize it.

In [ ]:
class MyAnalysis(object):
    def __init__(self, ml_model_1, ml_model_2):
        # your code here
        pass
    
    def run_all(self):
        # your code here
        # this should now call prepare_features for both
        # models, then train them, then validate them
        pass


# Initialize an analysis with two BDTs
# ma1 = your code here  
# ma1.run_all()
# Initialize an analysis with a BDT and a RandomForest
# ma2 = your code here
# ma2.run_all()

<html>
<details>
<summary style="color:orange; font-weight: bold;"> Click me for a hint </summary>
Your constructor should simply save the two models

```python
def __init__(self, ml_model_1, ml_model_2):
    # your code here
    self.ml_model_1 = ml_model_1
    self.ml_model_2 = ml_model_2
```
</details>
</html>

<html>
<details>
<summary style="color:orange; font-weight: bold;"> Click me for a hint </summary>
The full class should look like this:
    
```python
class MyAnalysis(object):
    def __init__(self, ml_model_1, ml_model_2):
        # your code here
        self.ml_model_1 = ml_model_1
        self.ml_model_2 = ml_model_2
    
    def run_all(self):
        # your code here
        # this should now call prepare_features for both
        # models, then train them, then validate them
        self.ml_model_1.prepare_features()
        self.ml_model_2.prepare_features()
        self.ml_model_1.train()
        self.ml_model_2.train()
        self.ml_model_1.validate()
        self.ml_model_2.validate()
```

Now you still need to find out how to initialize and run it.
</details>
</html>

<html>
<details>
<summary style="color:red; font-weight: bold;"> Click me for the solution </summary>
The full class should look like this:
    
```python
class MyAnalysis(object):
    def __init__(self, ml_model_1, ml_model_2):
        # your code here
        self.ml_model_1 = ml_model_1
        self.ml_model_2 = ml_model_2
    
    def run_all(self):
        # your code here
        # this should now call prepare_features for both
        # models, then train them, then validate them
        self.ml_model_1.prepare_features()
        self.ml_model_2.prepare_features()
        self.ml_model_1.train()
        self.ml_model_2.train()
        self.ml_model_1.validate()
        self.ml_model_2.validate()


# Initialize an analysis with two BDTs
ma1 = MyAnalysis(BDT(), BDT())  
ma1.run_all()
# Initialize an analysis with a BDT and a RandomForest
ma2 = MyAnalysis(BDT(), RandomForest())
ma2.run_all()
```

Now you still need to find out how to initialize and run it.
</details>
</html>